In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np 
from tensorflow import keras

In [3]:
data = pd.read_csv("insurance_data.csv")
data.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [5]:
X = data[['age', 'affordibility']]
y = data['bought_insurance']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

* We will maximize our epoch by increasing the number of epochs it iterate more into data so by reducing loss error we can achive high accuracy
* we will use sigmoid activation function as we want binary classification of the data
* we have use here BinaryCrossentropy as it is binary classification


In [19]:

model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(X_train_scaled.shape[1],), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train, epochs=4000)


Epoch 1/4000


d:\AIML Internship\Day 41-43 Nueral Nework\tf_env\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 675ms/step - accuracy: 0.5000 - loss: 0.7428
Epoch 2/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step - accuracy: 0.5000 - loss: 0.7424
Epoch 3/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.5000 - loss: 0.7420
Epoch 4/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5000 - loss: 0.7416
Epoch 5/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.5000 - loss: 0.7411
Epoch 6/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.5000 - loss: 0.7407
Epoch 7/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5000 - loss: 0.7403
Epoch 8/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5000 - loss: 0.7399
Epoch 9/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.5000 - loss: 0.7395
Epoch 10/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.5000 - loss: 0.7390
Epoch 11/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5000 - loss: 0.7386
Epoch 12/4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.5000 - l

* here we have achived good accuracy  on 4000 epoch

* we have 100% accuracy on test data

In [21]:
model.evaluate(X_test_scaled, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 1.0000 - loss: 0.3132


[0.3131769299507141, 1.0]

In [28]:
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


array([[0.7821655 ],
       [0.7296572 ],
       [0.7890334 ],
       [0.22989745],
       [0.4226227 ],
       [0.23719643]], dtype=float32)

In [29]:
y_test

9     1
25    1
8     1
21    0
0     0
12    0
Name: bought_insurance, dtype: int64

* in above we can see > 0.5 is 1 and < 0.5 is 0

In [30]:
coef, intercept = model.get_weights()
coef, intercept

(array([[4.0778127],
        [1.0599864]], dtype=float32),
 array([-2.2691214], dtype=float32))

# W1 = 4.077
# W2 = 1.059
# bias = -2.26

* Manual Gradient Descent

In [48]:
import math
def sigmoid(x):
    return 1/(1+np.exp(-x))
sigmoid(10)

np.float64(0.9999546021312976)

In [49]:
def prediction_function(age, affordibility):
    weighted_sum =coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

In [50]:
prediction_function(.22, 1)

array([0.42262268], dtype=float32)

In [47]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [60]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias

In [61]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,4000, 0.5146)

Epoch:0, w1:0.9736899318847281, w2:0.931388810977659, bias:-0.11748951666770448, loss:0.7428288579142563
Epoch:1, w1:0.9536535852311094, w2:0.8740290167758512, bias:-0.21881533456146035, loss:0.7072146449948487
Epoch:2, w1:0.9393731039296969, w2:0.8271852202997496, bias:-0.3053620401943441, loss:0.6814881914786812
Epoch:3, w1:0.9301932588998061, w2:0.7897792032048467, bias:-0.37884372361582785, loss:0.6633428084673968
Epoch:4, w1:0.9254091137248938, w2:0.7605726653866934, bias:-0.44108236820018304, loss:0.650742850709519
Epoch:5, w1:0.9243325693598607, w2:0.738313053647322, bias:-0.49384257986251556, loss:0.6420508089402462
Epoch:6, w1:0.926333296357235, w2:0.7218280753843739, bias:-0.5387319906498417, loss:0.6360356979531208
Epoch:7, w1:0.930858097563688, w2:0.7100747303660235, bias:-0.5771558825717441, loss:0.631816485354411
Epoch:8, w1:0.9374354910317362, w2:0.7021560855322683, bias:-0.6103083840841516, loss:0.6287844495353145
Epoch:9, w1:0.9456716791005845, w2:0.6973185496313956, b

(np.float64(4.024540501341598),
 np.float64(1.1689067465596255),
 np.float64(-2.356112458016952))

In [62]:
coef, intercept

(array([[4.0778127],
        [1.0599864]], dtype=float32),
 array([-2.2691214], dtype=float32))

* we can see value of w1, w2, bias remain almost same for both the model